# Project Objective: How to apply cross validation and compare diff models. 
   
    Please find below a summary of the overall approach.
    1. Chi-square test was performed to sigle out of the feature importance of the catagorical feature and it was concluded 
        that All catagorical features seems to have some kind of influence on the outcome.
    2. Following Features have been decided to be removed from model building and training as part of initial phase.
        -- Cabin : because most of the values (Aprox 70%) are missing
        -- Names:
        -- Ticket
    3. Model Accuracy has been tested with Cross validations score using following models.
          1.Decesion Trees Classifier
          2.RandomForest Classifier
          3.LogisticRegression
          4.Support Vector Classifier
          5.GradientBoosting Classifier
          6.XGBoost Classifier
          7.XGBoost Random Forest Classifier
    4. Based on 10 Crosss validation splits, it is found that the XGBoost Random Forest Classifier gave the better accuracy 
       score of Min Score of:75%, Maximum Score of:90% and Average Score of: 83%
    5. K-Fold Crosss validation was performed and extracted the best training and Testing sample out of 10 splits and 
       model was retrained.
    6. GridSearch Hyperparametr tuning approach was also tried to check if the model performance can further be increased.
## About Data
    The dataset has been taken from Kaggle Titanic Survivor_Prediction

In [1]:
import numpy as np
import pandas as pd

In [3]:
dataSet=pd.read_csv('train.csv')

In [4]:
dataSet.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Check the dataset for features and Label types
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# Check for Null Features and % of Null values across each Featur
# Features having Null values are 'Age','Cabin' and 'Embarked'
dataSet.isnull().sum()/len(dataSet)*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [7]:
# Fill in Null Values for 'Age' with mean value
dataSet.Age.fillna(np.round(dataSet.Age.mean()),inplace=True)
# Fill in Null Values for 'Embarked' with mode
dataSet.Embarked.fillna(dataSet.Embarked.mode()[0],inplace=True)

# Since there are 77% of missing values for the attribute Cabin, its better to discard this columns from model building as part of initial trial
# Lets also drop PassengerId,Name column from model building
dataSet1=dataSet.drop(columns=['PassengerId','Name','Cabin'])
dataSet1.head()
# Name           891 non-null object
# Embarked       889 non-null object
# Cabin          204 non-null object
# Sex            891 non-null object
# Ticket         891 non-null object

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,1,1,female,35.0,1,0,113803,53.1000,S
4,0,3,male,35.0,0,0,373450,8.0500,S


In [8]:
# Lets Make Age into Age_group  (Optional Step)
dataSet1.Age.describe()

count    891.000000
mean      29.758889
std       13.002570
min        0.420000
25%       22.000000
50%       30.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [9]:
def age_group(x):
    age_grp=''
    if (x >=0 and x <= 12):
        age_grp='Kid'
    elif (x >=13 and x <= 19):
        age_grp='Teenager'
    elif (x >=20 and x <= 55):
        age_grp='Adult'
    else:
        age_grp='Older'
    return  age_grp   

In [10]:
dataSet1['Age_group']=dataSet1.Age.apply(age_group)

In [11]:
dataSet1

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Age_group
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S,Adult
1,1,1,female,38.0,1,0,PC 17599,71.2833,C,Adult
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,Adult
3,1,1,female,35.0,1,0,113803,53.1000,S,Adult
4,0,3,male,35.0,0,0,373450,8.0500,S,Adult
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,S,Adult
887,1,1,female,19.0,0,0,112053,30.0000,S,Teenager
888,0,3,female,30.0,1,2,W./C. 6607,23.4500,S,Adult
889,1,1,male,26.0,0,0,111369,30.0000,C,Adult


In [12]:
# Lets try to figure out: Among all categorical features which features seems to be influncing most for predicting the outcome
# We will do Chisquare test for SELECTING TOP-N CATEGORICAL FEATURES. For this lets separate all categorical features and label
dataSet2=dataSet1.drop(columns=['Age','Fare','SibSp','Parch'])
dataSet2=dataSet2.astype(str)
dataSet2.head()

#From Below Chisquare Test it seems most influencing Categorical feature is 'Ticket' followed by 'Sex','Pclass'.
#However the scores are not that far apart and hence its not conclusive to leave out any acategorical feature at this point.
# We can possibly leave out Ticket Ticket and include Fare instead
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import OrdinalEncoder
features=dataSet2.drop(columns=['Survived'])
label=dataSet2.loc[:,['Survived']]
OE=OrdinalEncoder()
features_chi=OE.fit_transform(features)  # This is needed to encode each catagorical variable to integers values.
modelKBest=SelectKBest(score_func=chi2,k='all')
finalFeatures=modelKBest.fit_transform(features_chi,label)
print(features.columns)
print(modelKBest.scores_)

Index(['Pclass', 'Sex', 'Ticket', 'Embarked', 'Age_group'], dtype='object')
[5.44658660e+01 9.27024470e+01 2.87165547e+03 1.02025247e+01
 2.02415975e+00]


In [13]:
# Lets decide Final features and  label as below. Dropping out PassengerId,Name,Cabin, Ticket columns for the initial model. 
finalFeatures=dataSet.drop(columns=['PassengerId','Name','Cabin','Ticket','Survived'])
label=dataSet.loc[:,['Survived']]

In [14]:
# Lets apply OneHotEncoding for Categorical features
finalFeatures=pd.get_dummies(finalFeatures)
finalFeatures.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1


In [15]:
finalFeatures=finalFeatures.values
label=label.values

## Verify the performance of different models using Stratified-KFold Cross Validation 

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,classification_report,confusion_matrix
#from sklearn import metrics

def stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y):
    global df_model_selection
    
    skf = StratifiedKFold(n_splits, random_state=12,shuffle=True)
    
    weighted_f1_score = []
    #print(skf.split(X,y))
    for train_index, test_index in skf.split(X,y):
        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]
        
        
        model_obj.fit(X_train, y_train)##### HERE ###
        test_ds_predicted = model_obj.predict( X_test ) ##### HERE ####   
        #print( metrics.classification_report( y_test, test_ds_predicted ) )    
        weighted_f1_score.append(round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2))
        
    sd_weighted_f1_score = np.std(weighted_f1_score, ddof=1)
    range_of_f1_scores = "{}-{}".format(min(weighted_f1_score),max(weighted_f1_score))    
    df_model_selection = pd.concat([df_model_selection,pd.DataFrame([[process,model_name,sorted(weighted_f1_score),range_of_f1_scores,sd_weighted_f1_score]], columns =COLUMN_NAMES) ])
    

In [17]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRFClassifier,XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

COLUMN_NAMES = ["Process","Model Name", "F1 Scores","Range of F1 Scores","Std Deviation of F1 Scores"]
df_model_selection = pd.DataFrame(columns=COLUMN_NAMES)

process='Stratified-KFold'
n_splits = 10
X=finalFeatures
y=label

# 1.Naive Bayes
model_NB=BernoulliNB()
model_obj=model_NB
model_name='Naive Bayes'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 2.Logistic Regression
model_LR=LogisticRegression()
model_obj=model_LR
model_name='Logistic Regression'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 3.Decesion Tree Classifier
model_DTC=DecisionTreeClassifier()
model_obj=model_DTC
model_name='Decesion Tree Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 4.Random Forest Classifier
model_RFC=RandomForestClassifier()
model_obj=model_RFC
model_name='Random Forest Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 5.XGBoost Classifier
model_XGBC=XGBClassifier()
model_obj=model_XGBC
model_name='XGBoost Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 6.Gradient Boosting Classifier
model_GBC=GradientBoostingClassifier()
model_obj=model_GBC
model_name='Gradient Boosting Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 7.XGBoost Random Forest Classifier
model_XGBRFC=XGBRFClassifier()
model_obj=model_XGBRFC
model_name='XGBoost Random Forest Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 8.Support Vector Machine Classifier
model_SVC=SVC()
model_obj=model_SVC
model_name='Support Vector Machine Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)


# 9.SGD Classifier
model_sgd = OneVsRestClassifier(SGDClassifier())
model_obj=model_sgd
model_name='Stochastic Gradient Descent Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#10.Gausian Process Classifier
model_GPC = GaussianProcessClassifier()
model_obj=model_GPC
model_name='Gausian Process Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#11.Gausian Process Classifier
model_KNNC=KNeighborsClassifier()
model_obj=model_KNNC
model_name='K Nearst Neighbour Classifier'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#12 Linear Discriminant Analysis
model_LDA=LinearDiscriminantAnalysis()
model_obj=model_LDA
model_name='Linear Discriminant Analysis'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#Exporting the results to csv
#df_model_selection.to_csv("Model_statistics.csv",index = False)
df_model_selection

C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Softw

,Process,Model Name,F1 Scores,Range of F1 Scores,Std Deviation of F1 Scores
0,Stratified-KFold,Naive Bayes,"[0.74, 0.75, 0.78, 0.78, 0.78, 0.79, 0.79, 0.8...",0.74-0.82,0.025927
0,Stratified-KFold,Logistic Regression,"[0.76, 0.76, 0.78, 0.79, 0.79, 0.79, 0.81, 0.8...",0.76-0.83,0.023664
0,Stratified-KFold,Decesion Tree Classifier,"[0.74, 0.74, 0.75, 0.75, 0.79, 0.79, 0.8, 0.8,...",0.74-0.81,0.029364
0,Stratified-KFold,Random Forest Classifier,"[0.71, 0.77, 0.8, 0.81, 0.81, 0.82, 0.82, 0.83...",0.71-0.85,0.040332
0,Stratified-KFold,XGBoost Classifier,"[0.76, 0.78, 0.79, 0.79, 0.8, 0.81, 0.82, 0.83...",0.76-0.85,0.028304
0,Stratified-KFold,Gradient Boosting Classifier,"[0.78, 0.79, 0.8, 0.82, 0.83, 0.83, 0.84, 0.84...",0.78-0.88,0.029155
0,Stratified-KFold,XGBoost Random Forest Classifier,"[0.73, 0.81, 0.81, 0.82, 0.83, 0.83, 0.84, 0.8...",0.73-0.88,0.042111
0,Stratified-KFold,Support Vector Machine Classifier,"[0.56, 0.58, 0.59, 0.6, 0.6, 0.67, 0.67, 0.68,...",0.56-0.7,0.053800
0,Stratified-KFold,Stochastic Gradient Descent Classifier,"[0.52, 0.58, 0.63, 0.67, 0.72, 0.76, 0.76, 0.7...",0.52-0.81,0.099225
0,Stratified-KFold,Gausian Process Classifier,"[0.6, 0.66, 0.67, 0.68, 0.71, 0.71, 0.76, 0.78...",0.6-0.82,0.071181


### Conclusion sofar
#from above Cross validation results it is understood that Gradient Boosting Classifier is giving better perfprmance than others classifiers


In [18]:
# Now lets try to get the Scores using StratifiedKFold Cross Validation

#Initialize the algo
model=GradientBoostingClassifier()

#Initialize StratifiedKFold Method
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, 
              random_state=1,
              shuffle=True)

#Initialize For Loop 

i=0
for train,test in kfold.split(finalFeatures,label):
    i = i+1
    X_train,X_test = finalFeatures[train],finalFeatures[test]
    y_train,y_test = label[train],label[test]
    
    model.fit(X_train,y_train)
    test_ds_predicted=model.predict(X_test)
    train_ds_predicted=model.predict(X_train)
    
    test_f1_score=round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2)
    train_f1_score=round(f1_score(y_true=y_train, y_pred=train_ds_predicted , average='weighted'),2)
    
    #print("Train Score: {}, Test score: {}, for Sample Split: {}".format(model.score(X_train,y_train),model.score(X_test,y_test),i))
    print("Train f1-Score: {}, Test f1-score: {}, for Sample Split: {}".format(train_f1_score,test_f1_score,i))
        

C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Train f1-Score: 0.88, Test f1-score: 0.84, for Sample Split: 1
Train f1-Score: 0.9, Test f1-score: 0.83, for Sample Split: 2
Train f1-Score: 0.9, Test f1-score: 0.81, for Sample Split: 3


C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Train f1-Score: 0.9, Test f1-score: 0.89, for Sample Split: 4
Train f1-Score: 0.89, Test f1-score: 0.73, for Sample Split: 5
Train f1-Score: 0.89, Test f1-score: 0.8, for Sample Split: 6


C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Train f1-Score: 0.9, Test f1-score: 0.79, for Sample Split: 7
Train f1-Score: 0.9, Test f1-score: 0.88, for Sample Split: 8
Train f1-Score: 0.89, Test f1-score: 0.85, for Sample Split: 9
Train f1-Score: 0.9, Test f1-score: 0.78, for Sample Split: 10


C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [19]:
#Lets extract the Train and Test sample for split 4
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, #n_splits should be equal to no of cv value in cross_val_score
              random_state=1,
              shuffle=True)
i=0
for train,test in kfold.split(finalFeatures,label):
    i = i+1
    if i == 4:
        X_train,X_test,y_train,y_test = finalFeatures[train],finalFeatures[test],label[train],label[test]

In [20]:
#Final Model
finalModel=GradientBoostingClassifier()
finalModel.fit(X_train,y_train)

test_ds_predicted=model.predict(X_test)
train_ds_predicted=model.predict(X_train)

test_f1_score=round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2)
train_f1_score=round(f1_score(y_true=y_train, y_pred=train_ds_predicted , average='weighted'),2)
print("Train f1-Score: {}, Test f1-score: {}".format(train_f1_score,test_f1_score))


train_score=np.round(finalModel.score(X_train,y_train),2)
test_score=np.round(finalModel.score(X_test,y_test),2)
print('Train Accuracy Score is:{} and  Test Accuracy Score:{}'.format(train_score,test_score))

Train f1-Score: 0.88, Test f1-score: 0.94
Train Accuracy Score is:0.9 and  Test Accuracy Score:0.89


C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [21]:
# Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_true=label, y_pred=finalModel.predict(finalFeatures))
CR=classification_report(y_true=label, y_pred=finalModel.predict(finalFeatures))
print('Confusion Matrix:\n',cm)
print('\n Classification Report:\n',CR)

Confusion Matrix:
 [[527  22]
 [ 69 273]]

 Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92       549
           1       0.93      0.80      0.86       342

    accuracy                           0.90       891
   macro avg       0.90      0.88      0.89       891
weighted avg       0.90      0.90      0.90       891



#### Lets try to see if we can improve the model performance by Hyper parameter Tuning using Randomized Grid Search

In [22]:
from sklearn.model_selection import RandomizedSearchCV
#min_child_weight
#colsample_bylevel
#colsample_bytree
#gamma
param_grid={
            #'booster':['gbtree','gblinear','dart'],
            'max_depth':[5,6,7,8,9,10,15,20,30],
            'n_estimators':[50,100,120,150,200],
            'learning_rate':[0.05,0.1,0.5,0.9,1],
            'loss':['deviance','exponential'],
            #'criterion':[0.2,0.4,0.6,0.8,1.0],
            #'colsample_bynode':[0.2,0.4,0.6,0.8,1.0],
            'subsample':[0.5,0.6,0.7,0.8,0.9,1]
            #'gamma':[0.1,0.3,0.5,0.8,1]
           }
model_GBC=GradientBoostingClassifier()
RS = RandomizedSearchCV(estimator=model_GBC,param_distributions=param_grid,cv=10,n_iter=10)

RS.fit(finalFeatures,label) 


C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Software\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Softw

RandomizedSearchCV(cv=10, estimator=GradientBoostingClassifier(),
                   param_distributions={'learning_rate': [0.05, 0.1, 0.5, 0.9,
                                                          1],
                                        'loss': ['deviance', 'exponential'],
                                        'max_depth': [5, 6, 7, 8, 9, 10, 15, 20,
                                                      30],
                                        'n_estimators': [50, 100, 120, 150,
                                                         200],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1]})

In [155]:
RS.best_score_

0.8249812734082397

In [156]:
RS.best_estimator_

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [157]:
#Final Model
finalModel=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
finalModel.fit(X_train,y_train)

test_ds_predicted=model.predict(X_test)
train_ds_predicted=model.predict(X_train)

test_f1_score=round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2)
train_f1_score=round(f1_score(y_true=y_train, y_pred=train_ds_predicted , average='weighted'),2)
print("Train f1-Score: {}, Test f1-score: {}".format(train_f1_score,test_f1_score))


train_score=np.round(finalModel.score(X_train,y_train),2)
test_score=np.round(finalModel.score(X_test,y_test),2)
print('Train Accuracy Score is:{} and  Test Accuracy Score:{}'.format(train_score,test_score))

Train f1-Score: 0.88, Test f1-score: 0.94
Train Accuracy Score is:0.92 and  Test Accuracy Score:0.92


### Making the Test data ready for Predictions

In [158]:
testData=pd.read_csv('~/Public/skbc/Kaggle/test.csv')
testData.isnull().sum()/len(dataSet)*100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             9.652076
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.112233
Cabin          36.700337
Embarked        0.000000
dtype: float64

In [159]:
testData.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [143]:
# Fill in Null Values for 'Age' with mean value
testData.Age.fillna(np.round(testData.Age.mean()),inplace=True)

# Fill in Null Values for 'Fare' with mean value
testData.Fare.fillna(np.round(testData.Fare.mean()),inplace=True)

# Lets make the Final features same as Train dataset
finalTestdata=testData.drop(columns=['PassengerId','Name','Cabin','Ticket'])

# Lets apply OneHotEncoding for Categorical features
finalTestdata=pd.get_dummies(finalTestdata).values

# Preddict the survivir based on test data
survivor=finalModel.predict(finalTestdata)

# Export the prediction as CSV file
prediction=testData[['PassengerId']]
prediction['Survived']=pd.DataFrame(survivor,columns=['Survived'])
#prediction.to_csv('D:\SarojOfficeWork\AI\Kaggle\TitanicSurviver\prediction.csv')
prediction.to_csv('prediction.csv')